In [36]:
import json, pandas as pd

def load_trace(path):
    with open(path, 'r') as f:
        data = json.load(f)
    if isinstance(data, dict) and 'traceEvents' in data:
        return data['traceEvents']
    elif isinstance(data, list):
        return data
    else:
        raise ValueError("Unrecognized trace format")

def build_dataframe(events):
    rows = []
    for e in events:
        dur = e.get('dur')
        if dur is None: continue
        cat = e.get('cat')
        name = e.get('name','')
        args = e.get('args') or {}
        if cat == 'Node':
            prov = args.get('provider','CPUExecutionProvider')
            if prov == 'WebGpuExecutionProvider':
                prov = 'WebGPU-CPU'
            op   = args.get('op_name', name)
            rows.append({'ep':prov.replace('ExecutionProvider',''),
                         'op':op,'shader':None,'shape':None,'dur':dur})
        elif cat == 'Kernel':
            rows.append({'ep':'CUDA/ROCm','op':name,'shader':None,'shape':None,'dur':dur})
        elif cat == 'Api':
            parts = name.split('&')
            op     = parts[1] if len(parts)>=2 else name
            shader = parts[2] if len(parts)>=3 else None
            shape  = args.get('shapes')
            rows.append({'ep':'WebGPU-GPU','op':op,'shader':shader,'shape':shape,'dur':dur})
    return pd.DataFrame(rows)

def summarize(df, ep_name, group_keys):
    sub = df[df.ep==ep_name].copy()
    if sub.empty:
        print(f"No data for {ep_name}")
        return
    total = sub.dur.sum()
    keys  = [k for k in group_keys if k in sub.columns]
    agg = (sub.groupby(keys).dur
             .agg(count='size', total='sum')
             .reset_index()
          )
    agg['avg']     = agg.total/agg['count']
    agg['pct']     = 100*agg.total/total
    agg = agg.sort_values('total', ascending=False)
    agg['cum_pct'] = agg.pct.cumsum()
    print(f"\n--- {ep_name} by {'+'.join(keys)} ---")
    print(agg.round(2).to_string(index=False))

def show(path, shape=False):
    # Use raw string or double backslashes: r"C:\dev\ort_env\file.json"
    events = load_trace(path)
    df = build_dataframe(events)
    for ep in df.ep.unique():
        if ep != 'WebGPU-GPU':
            summarize(df, ep, ['op'])
        else:
            summarize(df, 'WebGPU-GPU', ['op'] + (['shape'] if shape else []))
            summarize(df, 'WebGPU-GPU', ['shader'] + (['shape'] if shape else []))


In [37]:
show("C:\dev\ort_env\yolo11n_desktop_cpu.json")

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\vrparikh\AppData\Local\Temp\ipykernel_23988\326119538.py:1: SyntaxWarning: invalid escape sequence '\d'
  show("C:\dev\ort_env\yolo11n_desktop_cpu.json")



--- CPU by op ---
           op  count  total     avg   pct  cum_pct
         Conv   1848 204154  110.47 27.34    27.34
ReorderOutput   1848 153579   83.11 20.56    47.90
 ReorderInput   1659 106231   64.03 14.22    62.12
    QuickGelu   1617  92931   57.47 12.44    74.57
      Softmax     42  85002 2023.86 11.38    85.95
       Concat    483  40572   84.00  5.43    91.38
        Split    252  19791   78.54  2.65    94.03
          Add    336  15708   46.75  2.10    96.14
       Resize     42  12242  291.48  1.64    97.77
      Reshape    168   4338   25.82  0.58    98.36
  FusedMatMul     42   3533   84.12  0.47    98.83
      MaxPool     63   2335   37.06  0.31    99.14
      Sigmoid     21   1908   90.86  0.26    99.40
    Transpose     21   1547   73.67  0.21    99.60
          Sub     42   1428   34.00  0.19    99.80
          Mul     21    843   40.14  0.11    99.91
          Div     21    688   32.76  0.09   100.00


In [38]:
show(r"C:\dev\ort_env\yolo11n_desktop_webgpu.json")


--- WebGPU-CPU by op ---
       op  count  total    avg   pct  cum_pct
     Conv   1848 513038 277.62 48.98    48.98
      Mul   1659 155401  93.67 14.83    63.81
  Sigmoid   1638 113005  68.99 10.79    74.60
   Concat    483  75650 156.63  7.22    81.82
Transpose    357  55114 154.38  5.26    87.08
    Split    231  41656 180.33  3.98    91.06
      Add    336  31766  94.54  3.03    94.09
  Softmax     42  14342 341.48  1.37    95.46
   Resize     42  13009 309.74  1.24    96.70
  MaxPool     63  11612 184.32  1.11    97.81
   MatMul     42   8550 203.57  0.82    98.63
  Reshape    168   5027  29.92  0.48    99.11
    Slice     42   4559 108.55  0.44    99.54
      Sub     42   2893  68.88  0.28    99.82
      Div     21   1910  90.95  0.18   100.00

--- WebGPU-GPU by op ---
       op  count  total    avg   pct  cum_pct
     Conv   3696 255784  69.21 62.82    62.82
      Mul   1659  37679  22.71  9.25    72.08
  Sigmoid   1638  28908  17.65  7.10    79.18
   Concat    483  24242  50.

In [39]:
show(r'C:\dev\perf_results\pixel\image_perf_and_logs\yolo11n_cpu_2025-05-01_17-49-54.json')


--- CPU by op ---
         op  count  total     avg   pct  cum_pct
       Conv    170 131057  770.92 76.12    76.12
  QuickGelu    154  22009  142.92 12.78    88.91
     Concat     46   7989  173.67  4.64    93.55
    Softmax      4   4708 1177.00  2.73    96.28
      Split     24   1852   77.17  1.08    97.36
FusedMatMul      4   1408  352.00  0.82    98.18
  FusedConv      6    955  159.17  0.55    98.73
        Add     26    678   26.08  0.39    99.12
    Sigmoid      2    516  258.00  0.30    99.42
     Resize      4    349   87.25  0.20    99.63
    MaxPool      6    191   31.83  0.11    99.74
  Transpose      2    160   80.00  0.09    99.83
    Reshape     16    149    9.31  0.09    99.92
        Sub      4     60   15.00  0.03    99.95
        Mul      2     43   21.50  0.02    99.98
        Div      2     39   19.50  0.02   100.00


In [40]:
show(r"C:\dev\perf_results\pixel\image_perf_and_logs\yolo11n_webgpu_2025-05-01_17-48-54.json")


--- WebGPU-CPU by op ---
       op  count  total     avg   pct  cum_pct
     Conv    176 327689 1861.87 56.90    56.90
   Concat     46  53357 1159.93  9.26    66.16
   MatMul      4  39793 9948.25  6.91    73.07
Transpose     34  31173  916.85  5.41    78.48
    Split     22  26498 1204.45  4.60    83.09
      Mul    158  25009  158.28  4.34    87.43
  Sigmoid    156  16624  106.56  2.89    90.31
  Softmax      4  15208 3802.00  2.64    92.95
   Resize      4  12669 3167.25  2.20    95.15
  MaxPool      6   9992 1665.33  1.73    96.89
    Slice      4   6973 1743.25  1.21    98.10
      Add     32   5576  174.25  0.97    99.07
      Sub      4   2734  683.50  0.47    99.54
      Div      2   2485 1242.50  0.43    99.97
  Reshape     16    145    9.06  0.03   100.00

--- WebGPU-GPU by op ---
       op  count  total    avg   pct  cum_pct
     Conv    352 177412 504.01 70.02    70.02
      Mul    158  17727 112.20  7.00    77.01
Transpose     34  14834 436.29  5.85    82.87
  Sigmoid   